In [21]:
# Initial imports
import pandas as pd
import hvplot.pandas
from path import Path
import plotly.express as px
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
import matplotlib as plt
from sklearn.datasets import make_blobs
import sklearn as skl
import tensorflow as tf
import numpy as np

In [8]:
file_path = "./cleaned_music_data.csv"
music_df = pd.read_csv(file_path, index_col=0, encoding = "ISO-8859-1")
music_df

,year,acousticness,danceability,duration_ms,energy,explicit,instrumentalness,loudness,popularity,tempo
7606,1960,0.547000,0.274,179693,0.348,0,0.013300,-8.631,76,87.430
7607,1960,0.829000,0.421,196133,0.285,0,0.000002,-9.430,70,85.861
7608,1960,0.764000,0.508,175987,0.287,0,0.000000,-12.472,69,154.759
7609,1960,0.733000,0.579,131733,0.502,0,0.000000,-7.570,69,76.816
7610,1960,0.699000,0.743,139200,0.799,0,0.000000,-5.466,62,116.112
...,...,...,...,...,...,...,...,...,...,...
170448,2019,0.226000,0.757,159547,0.688,0,0.000000,-4.546,71,155.994
170449,2019,0.001210,0.449,247613,0.825,0,0.000000,-5.177,60,135.934
170450,2019,0.016700,0.616,203555,0.469,0,0.000000,-7.380,63,135.035
170451,2019,0.000755,0.766,196875,0.649,0,0.545000,-13.051,59,127.996


In [13]:
X = music_df
# standardize all of the data from the X DataFrame.
X_scaled = StandardScaler().fit_transform(X)
X_scaled

array([[-1.70924941,  0.56589094, -1.59155199, ...,  0.27945633,
         2.26553439, -1.07297153],
       [-1.70924941,  1.42053226, -0.74588265, ...,  0.12303705,
         1.86817967, -1.12521682],
       [-1.70924941,  1.22354047, -0.24538447, ..., -0.47249165,
         1.80195388,  1.16898095],
       ...,
       [ 1.70802095, -1.04125902,  0.37592362, ...,  0.5243631 ,
         1.40459917,  0.51220199],
       [ 1.70802095, -1.08958263,  1.23885151, ..., -0.58584178,
         1.13969602,  0.27781408],
       [ 1.70802095, -0.38573095,  1.31939145, ...,  0.02593573,
         1.07347024,  1.20977157]])

In [24]:
# Use PCA to reduce the dimensions of the X DataFrame down to three principal components.
pca = PCA(n_components=6)
X_pca = pca.fit_transform(X_scaled)

# pca dataframe
pcs_df = pd.DataFrame(
    data = X_pca,
    columns = ["PC 1", "PC 2", "PC 3","PC 4", "PC 5", "PC 6"]).set_index(X.index , drop=False)

pcs_df

,PC 1,PC 2,PC 3,PC 4,PC 5,PC 6
7606,0.861477,0.692502,-0.082837,-0.596566,-1.343942,-0.075774
7607,1.330673,1.015585,-0.659979,-0.309456,-1.214085,0.287560
7608,1.140718,0.311854,-0.445333,-1.659232,-0.390651,1.186192
7609,0.490990,0.821486,-1.407509,-0.021420,-1.153203,-0.423582
7610,-0.496450,-0.265731,-1.538024,-0.226049,-0.534984,-0.194088
...,...,...,...,...,...,...
170448,-2.589483,0.820714,-0.038523,-1.478119,-0.861219,0.194022
170449,-2.311489,0.000916,1.114372,-0.766569,-0.967616,-0.097318
170450,-1.763073,0.938258,0.348945,-0.928561,-0.929325,0.176593
170451,-1.176299,1.254902,0.982240,-0.422841,-0.099709,-1.358124


In [25]:
# Create an elbow curve to find the best value for K, and use the pcs_df DataFrame.
inertia = []
k = list(range(1, 11))

# Calculate the inertia for the range of K values
for i in k:
    km = KMeans(n_clusters=i, random_state=0)
    km.fit(pcs_df)
    inertia.append(km.inertia_)

# Create the elbow curve
elbow_data = {"k": k, "inertia": inertia}
df_elbow = pd.DataFrame(elbow_data)
df_elbow.hvplot.line(x="k", y="inertia", xticks=k, title="Elbow Curve")

:Curve   [k]   (inertia)

In [26]:
## Create an elbow curve to find the best value for K, and use the pcs_df DataFrame.
inertia = []
k = list(range(1, 11))

# Calculate the inertia for the range of K values
for i in k:
    km = KMeans(n_clusters=i, random_state=0)
    km.fit(pcs_df)
    inertia.append(km.inertia_)

# Create the elbow curve
elbow_data = {"k": k, "inertia": inertia}
df_elbow = pd.DataFrame(elbow_data)
df_elbow.hvplot.line(x="k", y="inertia", xticks=k, title="Elbow Curve")

:Curve   [k]   (inertia)

In [28]:
# run the K-means algorithm to predict the K clusters for the cryptocurrencies’ data. Use the pcs_df to run the K-means algorithm.
# Initialize the K-means model
model = KMeans(n_clusters=5, random_state=0)

# Fit the model
model.fit(pcs_df)

# Predict clusters
predictions = model.predict(pcs_df)

# Add the predicted class columns
pcs_df["Class"] = model.labels_
pcs_df

,PC 1,PC 2,PC 3,PC 4,PC 5,PC 6,Class
7606,0.861477,0.692502,-0.082837,-0.596566,-1.343942,-0.075774,4
7607,1.330673,1.015585,-0.659979,-0.309456,-1.214085,0.287560,4
7608,1.140718,0.311854,-0.445333,-1.659232,-0.390651,1.186192,4
7609,0.490990,0.821486,-1.407509,-0.021420,-1.153203,-0.423582,4
7610,-0.496450,-0.265731,-1.538024,-0.226049,-0.534984,-0.194088,0
...,...,...,...,...,...,...,...
170448,-2.589483,0.820714,-0.038523,-1.478119,-0.861219,0.194022,0
170449,-2.311489,0.000916,1.114372,-0.766569,-0.967616,-0.097318,0
170450,-1.763073,0.938258,0.348945,-0.928561,-0.929325,0.176593,0
170451,-1.176299,1.254902,0.982240,-0.422841,-0.099709,-1.358124,0


In [32]:
clustered_df = music_df.merge(pcs_df, left_index= True, right_index=True)

clustered_df = clustered_df[['year','acousticness','danceability','duration_ms','energy','explicit','instrumentalness','loudness','popularity','tempo', 'Class']]
print(clustered_df.shape)
clustered_df

(118720, 11)


,year,acousticness,danceability,duration_ms,energy,explicit,instrumentalness,loudness,popularity,tempo,Class
7606,1960,0.547000,0.274,179693,0.348,0,0.013300,-8.631,76,87.430,4
7607,1960,0.829000,0.421,196133,0.285,0,0.000002,-9.430,70,85.861,4
7608,1960,0.764000,0.508,175987,0.287,0,0.000000,-12.472,69,154.759,4
7609,1960,0.733000,0.579,131733,0.502,0,0.000000,-7.570,69,76.816,4
7610,1960,0.699000,0.743,139200,0.799,0,0.000000,-5.466,62,116.112,0
...,...,...,...,...,...,...,...,...,...,...,...
170448,2019,0.226000,0.757,159547,0.688,0,0.000000,-4.546,71,155.994,0
170449,2019,0.001210,0.449,247613,0.825,0,0.000000,-5.177,60,135.934,0
170450,2019,0.016700,0.616,203555,0.469,0,0.000000,-7.380,63,135.035,0
170451,2019,0.000755,0.766,196875,0.649,0,0.545000,-13.051,59,127.996,0


In [34]:
clustered_df.to_csv('clustered_music_data.csv', sep=',')